In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
df = pd.read_parquet('../DATA/email.parquet')

In [ ]:
X = df.drop(columns=['email_type'])
y = df['email_type']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=100)

In [ ]:
model = LogisticRegression()
model.fit(X_train, y_train)